In [6]:
%matplotlib inline
import numpy as np
import json
import seaborn as sns
import pandas as pd
import os
import glob
import SimpleITK as itk
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
sns.set_style("whitegrid")
from src.utils import save_json, subfiles, join, maybe_mkdir_p, reconstruct_seg_df_from_json, reconstruct_calib_df_from_json, reconstruct_UED_df_from_json



# Statistics for different studies

In [17]:
from statsmodels.stats.multitest import multipletests
groups = ['Baseline','No TTA', 'MC Dropout','Ensemble','Snapshots', "Complex" ,'PhiSeg']
from scipy.stats import wilcoxon
def calcualte_test(x, y, method = wilcoxon):
    # method could also be pearsonr
    nas = np.logical_or(np.isnan(x), np.isnan(y)) # remove nan

    return method(x[~nas], y[~nas])

# if data is screwed
def bootstrap_confidence_interval(data, func=np.mean, confidence=95, size=10000):
    """
    func could be np.mean/np.median
    """
    
    bs_replicates = np.empty(size)
    
    # Create bootstrap replicates as much as size
    for i in range(size):
        # Create a bootstrap sample
        bs_sample = np.random.choice(data,size=len(data))
        # Get bootstrap replicate and append to bs_replicates
        bs_replicates[i] = func(bs_sample)
        
    conf_interval = np.percentile(bs_replicates,[100-confidence,confidence])
    np.round(func(bs_replicates), 2), tuple(np.round(conf_interval,2))
    m = np.round(func(bs_replicates), 2)
    ci0, ci1 = np.round(conf_interval,2)
    #return_str = str(m)+'('+str(ci0)+'-'+ str(ci1)+')'
    return_str = f'{m:.2f}({ci0:.2f}-{ci1:.2f})'
    return return_str


def print_tests(data, exp):
    """
    !!!baseline data frist!!!
    """
    print(exp)
    for gtv in ["GTV-T", "GTV-N"]:
        print(gtv)
        pvalues = [] 
        means = []
        for group in groups:
            #print("----------------------")
            #
            test_all_data = data[((data["Ucertainty estimation methods"]==group) | (data["Ucertainty estimation methods"]=="Baseline")) & (data["GTV"]==gtv) ]
            base = test_all_data[(test_all_data["Ucertainty estimation methods"]=="Baseline")][exp].astype({exp: float})
            test = test_all_data[(test_all_data["Ucertainty estimation methods"]==group)][exp].astype({exp: float})
            means.append(str(bootstrap_confidence_interval(test.dropna())))
            if group == "Baseline":
                #pvalues.append(1)
                continue

            s, p = calcualte_test(base.values,test.values)
            # = np.round(p, 5)
            #print(type(p), p)
            pvalues.append(p)
        print("pvalues before correction: ", pvalues)
        corrected_pvalues = multipletests(pvalues)
        #print("pvalues after correction:", corrected_pvalues[0])
        corrected = [1]+ list(corrected_pvalues[1])
        print("pvalues after correction:", corrected)

        for i, p in enumerate(corrected):
            if p < 0.001:
                print(means[i]+"***")
            elif p < 0.01:
                print(means[i]+"**")
            elif p < 0.05:
                print(means[i]+"*")
            else:
                #print(str(np.round(p, 3)))
                p = np.round(p, 4)
                print(means[i]+str(p))


## Segmentation performance comparision 
Internal

In [18]:
## directorys for internal  probability maps 

DROP2_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
NNUNET_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
PH_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
PH_GAMM_PMAP="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg_gamma__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"

studies = [
join(NNUNET_PMAPS,'f0_tta'),
join(NNUNET_PMAPS,'f0'),
join(DROP2_PMAPS,'f0_mc10_tta')
,join(NNUNET_PMAPS,'f01234_tta')
,join(DROP2_PMAPS,'f0_tta_snap')
,join(DROP2_PMAPS,'f01234_mc10_tta_snap')
,join(PH_PMAPS,'f01234_tta')
]

groups = ['Baseline','No TTA', 'MC Dropout','Ensemble','Snapshots', "Complex" ,'PhiSeg']
for i, study in enumerate(studies):
    gtvt_seg, gtvn_seg  = reconstruct_seg_df_from_json(study)
    df_seg_t = pd.DataFrame.from_dict(gtvt_seg).T
    df_seg_n = pd.DataFrame.from_dict(gtvn_seg).T

    dft = df_seg_t
    dfn = df_seg_n

    dft['Ucertainty estimation methods'] = groups[i]
    dfn['Ucertainty estimation methods'] = groups[i]

    dft.index.names = ['PatientID']
    dfn.index.names = ['PatientID']
    dft= dft.reset_index(drop=False)
    dfn= dfn.reset_index(drop=False)

    if i == 0:
        dft_all_correct = dft
        dfn_all_correct = dfn
    else:
        dft_all_correct = pd.concat([dft_all_correct, dft])
        dfn_all_correct = pd.concat([dfn_all_correct, dfn])

all_seg_internal = pd.concat([dft_all_correct, dfn_all_correct])
all_seg_internal= all_seg_internal.reset_index(drop=False)


In [20]:
exp= 'DSC' 
#exp= 'HD95 (mm)'
#exp= 'Mean Surface Distance (mm)'
#exp= 'Surface Dice 2mm'

print_tests(all_seg_internal, exp)

DSC
GTV-T
pvalues before correction:  [0.008971217549269767, 0.13180683322738812, 0.002308022296133644, 0.04035775996940854, 0.4515773027033496, 0.00772192657235769]
pvalues after correction: [1, 0.03801793805373229, 0.24624062516934372, 0.013768474742922695, 0.11625276619168756, 0.4515773027033496, 0.03801793805373229]
0.68(0.64-0.71)1
0.67(0.63-0.70)*
0.67(0.64-0.71)0.2462
0.69(0.65-0.72)*
0.67(0.63-0.70)0.1163
0.67(0.63-0.71)0.4516
0.63(0.58-0.67)*
GTV-N
pvalues before correction:  [0.011697936760166063, 0.10313682673243209, 0.005390649509551264, 0.3744737555342701, 0.21150382409725055, 0.005199335321500139]
pvalues after correction: [1, 0.04597708103169247, 0.27859595268392423, 0.03079331575494367, 0.3782737805867404, 0.3782737805867404, 0.03079331575494367]
0.63(0.57-0.69)1
0.63(0.57-0.69)*
0.63(0.57-0.69)0.2786
0.64(0.58-0.70)*
0.62(0.56-0.68)0.3783
0.64(0.58-0.70)0.3783
0.64(0.58-0.70)*


External

In [49]:
## directorys for external probability maps 

DROP2_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"
NNUNET_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"
PH_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"
PH_GAMM_PMAP="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg_gamma__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"

studies = [
join(NNUNET_PMAPS,'f0_tta'),
join(NNUNET_PMAPS,'f0'),
join(DROP2_PMAPS,'f0_mc10_tta')
,join(NNUNET_PMAPS,'f01234_tta')
,join(DROP2_PMAPS,'f0_tta_snap')
,join(DROP2_PMAPS,'f01234_mc10_tta_snap')
,join(PH_PMAPS,'f01234_tta')
]
#,join(PH_GAMM_PMAP,'f0_tta')]


groups = ['Baseline', 'No TTA','MC Dropout','Ensemble','Snapshots', "Complex" ,'PhiSeg']
for i, study in enumerate(studies):
    gtvt_seg, gtvn_seg  = reconstruct_seg_df_from_json(study)
    df_seg_t = pd.DataFrame.from_dict(gtvt_seg).T
    df_seg_n = pd.DataFrame.from_dict(gtvn_seg).T

    dft = df_seg_t
    dfn = df_seg_n

    dft['Ucertainty estimation methods'] = groups[i]
    dfn['Ucertainty estimation methods'] = groups[i]

    dft.index.names = ['PatientID']
    dfn.index.names = ['PatientID']
    dft= dft.reset_index(drop=False)
    dfn= dfn.reset_index(drop=False)

    if i == 0:
        dft_all_correct = dft
        dfn_all_correct = dfn
    else:
        dft_all_correct = pd.concat([dft_all_correct, dft])
        dfn_all_correct = pd.concat([dfn_all_correct, dfn])


all_seg_external = pd.concat([dft_all_correct, dfn_all_correct])
all_seg_external= all_seg_external.reset_index(drop=False)

# exp= 'DSC' 
# exp= 'HD95 (mm)'
# exp= 'Mean Surface Distance (mm)'
exp= 'Surface Dice 2mm'
print_tests(all_seg_external, exp)

Surface Dice 2mm
GTV-T
pvalues before correction:  [0.049309693183975635, 0.10734313335396876, 2.0498904903972427e-08, 0.0004754376320679146, 1.0957672261788388e-07, 0.3491034180382393]
pvalues after correction: [1, 0.14075463587469092, 0.2031637184296896, 1.229934231207582e-07, 0.0019003947124421267, 5.478834930188512e-07, 0.3491034180382393]
0.55(0.53-0.57)
0.53(0.50-0.55)
0.54(0.52-0.57)
0.58(0.55-0.60)***
0.55(0.53-0.58)**
0.58(0.56-0.60)***
0.53(0.51-0.56)
GTV-N
pvalues before correction:  [0.004563590323981993, 0.16680243408877604, 3.0479477750938654e-07, 0.49053175526994763, 0.12838190966958027, 3.11428497349035e-11]
pvalues after correction: [1, 0.018129782894158668, 0.337815964505617, 1.523972958548652e-06, 0.49053175526994763, 0.337815964505617, 1.8685709839487287e-10]
0.65(0.62-0.67)
0.63(0.61-0.66)*
0.63(0.60-0.66)
0.68(0.65-0.70)***
0.63(0.61-0.66)
0.67(0.64-0.69)
0.61(0.58-0.63)***


## Calibarion performance comparision 
Internal

In [34]:

groups = ['Baseline','No TTA', 'MC Dropout','Ensemble','Snapshots',"Complex", 'PhiSeg' ]
## directorys for main internal probability maps 

DROP2_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
NNUNET_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
PH_PMAP="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
PH_GAMM_PMAP="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg_gamma__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"

studies = [
join(NNUNET_PMAPS,'f0_tta'),
join(NNUNET_PMAPS,'f0'),
join(DROP2_PMAPS,'f0_mc10_tta')
,join(NNUNET_PMAPS,'f01234_tta')
#,join(NNUNET_PMAPS,'f012456_tta')
,join(DROP2_PMAPS,'f0_tta_snap')
,join(DROP2_PMAPS,'f01234_mc10_tta_snap')
,join(PH_PMAP,'f01234_tta')

]
#,join(PH_GAMM_PMAP,'f0_tta')]

for i, study in enumerate(studies):
    gtvt_calib, gtvn_calib  = reconstruct_calib_df_from_json(study)
    df_cal_t = pd.DataFrame.from_dict(gtvt_calib).T
    df_cal_n = pd.DataFrame.from_dict(gtvn_calib).T

    df_cal_t['GTV'] = 'GTV-T'
    
    df_cal_n['GTV'] = 'GTV-N'

    dft = df_cal_t
    dfn = df_cal_n
    
    dft['Ucertainty estimation methods'] = groups[i]
    dfn['Ucertainty estimation methods'] = groups[i]
    dft.index.names = ['PatientID']
    dfn.index.names = ['PatientID']
    dft= dft.reset_index(drop=False)
    dfn= dfn.reset_index(drop=False)

    if i == 0:
        dft_all = dft
        dfn_all = dfn
    else:
        dft_all = pd.concat([dft_all, dft])
        dfn_all = pd.concat([dfn_all, dfn])

all_calib_internal = pd.concat([dft_all, dfn_all])
all_calib_internal= all_calib_internal.reset_index(drop=True)

## directorys for external probability maps 

DROP2_PMAPS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"
NNUNET_PMAPS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"
PH_PMAPS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"
PH_GAMM_PMAPS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg_gamma__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct"

studies = [
join(NNUNET_PMAPS_EXT,'f0_tta'),#X8
join(NNUNET_PMAPS_EXT,'f0'), #X1
join(DROP2_PMAPS_EXT,'f0_mc10_tta') #X80
,join(NNUNET_PMAPS_EXT,'f01234_tta') #X40
#,join(NNUNET_PMAPS_EXT,'f012456_tta')
,join(DROP2_PMAPS_EXT,'f0_tta_snap') #X40
,join(DROP2_PMAPS_EXT,'f01234_mc10_tta_snap') #X2000
,join(PH_PMAPS_EXT,'f01234_tta') #X40
]

for i, study in enumerate(studies):
    gtvt_calib, gtvn_calib  = reconstruct_calib_df_from_json(study)
    df_cal_t = pd.DataFrame.from_dict(gtvt_calib).T
    df_cal_n = pd.DataFrame.from_dict(gtvn_calib).T

    df_cal_t['GTV'] = 'GTV-T'
    df_cal_n['GTV'] = 'GTV-N'
    
    dft = df_cal_t
    dfn = df_cal_n

    dft['Ucertainty estimation methods'] = groups[i]
    dfn['Ucertainty estimation methods'] = groups[i]
    dft.index.names = ['PatientID']
    dfn.index.names = ['PatientID']
    dft= dft.reset_index(drop=False)
    dfn= dfn.reset_index(drop=False)

    if i == 0:
        dft_all = dft
        dfn_all = dfn
    else:
        dft_all = pd.concat([dft_all, dft])
        dfn_all = pd.concat([dfn_all, dfn])
        dft_all= dft_all.reset_index(drop=True)
        dfn_all= dfn_all.reset_index(drop=True)


all_calib_external = pd.concat([dft_all, dfn_all])
all_calib_external= all_calib_external.reset_index(drop=True)


internal

In [37]:
# exp= 'Brier Score' 
exp= 'Expected Calibration Error'

print_tests(all_calib_internal, exp)

Expected Calibration Error
GTV-T
pvalues before correction:  [1.218119098708288e-17, 0.02263847169630356, 3.369310934424041e-14, 2.571044027114986e-09, 5.853697609433029e-14, 1.3533873592059003e-10]
pvalues after correction: [1, 7.308714592249727e-17, 0.02263847169630356, 1.684655467211907e-13, 5.142088047619705e-09, 2.341479043773006e-13, 4.0601620770682035e-10]
0.28(0.26-0.31)
0.36(0.33-0.39)***
0.26(0.24-0.28)*
0.24(0.22-0.27)***
0.23(0.20-0.25)***
0.21(0.18-0.23)***
0.18(0.15-0.22)***
GTV-N
pvalues before correction:  [1.3354633299502592e-11, 0.9978067952264031, 2.0563938125435043e-09, 7.81531520340445e-05, 3.293749895989552e-11, 2.182312842251161e-11]
pvalues after correction: [1, 8.012779979434034e-11, 0.9978067952264031, 6.169181424944246e-09, 0.00015630019615291615, 1.317499958330728e-10, 1.0911564210779555e-10]
0.26(0.23-0.30)
0.31(0.27-0.35)***
0.25(0.22-0.28)
0.24(0.20-0.27)***
0.22(0.20-0.25)***
0.21(0.18-0.23)***
0.17(0.13-0.21)***


external

In [39]:
# exp= 'Brier Score' 
exp= 'Expected Calibration Error'

print_tests(all_calib_external, exp)

Expected Calibration Error
GTV-T
pvalues before correction:  [8.114841775971355e-36, 6.96010760470907e-09, 4.239576622725384e-27, 1.3607751916168863e-25, 3.515473118108575e-35, 2.5860796143776974e-28]
pvalues after correction: [1, 4.868905065582814e-35, 6.96010760470907e-09, 1.2718729868176152e-26, 2.7215503832337726e-25, 1.7577365590542875e-34, 1.034431845751079e-27]
0.36(0.34-0.38)
0.45(0.43-0.47)***
0.32(0.30-0.34)***
0.30(0.28-0.31)***
0.28(0.26-0.29)***
0.24(0.22-0.25)***
0.23(0.21-0.25)***
GTV-N
pvalues before correction:  [1.843470407957155e-30, 0.002466902172726311, 7.192241469673306e-21, 2.862082052875858e-17, 4.3677073664399924e-26, 2.3746676417965406e-16]
pvalues after correction: [1, 1.106082244774293e-29, 0.002466902172726311, 2.876896587869322e-20, 8.586246158627573e-17, 2.1838536832199963e-25, 4.749335283593081e-16]
0.35(0.32-0.37)
0.41(0.39-0.44)***
0.33(0.31-0.35)**
0.31(0.28-0.33)***
0.29(0.27-0.31)***
0.27(0.25-0.29)***
0.27(0.25-0.30)***


## Uncertainty Error Overlap Degree (UED) comparision
Internal

In [40]:
## directorys for main internal probability maps 

NNUNET_UREGIONS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"
DROP1_UREGIONS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout1__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"
DROP2_UREGIONS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"
DROP3_UREGIONS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout3__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"
DROP5_UREGIONS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout5__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"

PHISEG_UREGIONS = "/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"
PHISEG_GAMMA_UREGIONS= "/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg_gamma__nnUNetPlansv2.1/u_regions/Task901_AUH/imagesTs"

studies = [
join(NNUNET_UREGIONS,'f0_tta'),
join(NNUNET_UREGIONS,'f0'),
join(DROP2_UREGIONS,'f0_mc10_tta')
,join(NNUNET_UREGIONS,'f01234_tta')
#,join(NNUNET_PMAPS,'f012456_tta')
,join(DROP2_UREGIONS,'f0_tta_snap')
,join(DROP2_UREGIONS,'f01234_mc10_tta_snap')
,join(PHISEG_UREGIONS,'f01234_tta')

]


def compute_best_ued(studies):

    # Define the base directory
    for index, group in enumerate(groups):

        base_dir = studies[index]

        # Define the subdirectories to iterate through
        #subdirs = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"] #best th leads to results without 
        subdirs = ["7"]
        # Initialize a dictionary to store the results for each patient

        results = [] 
        # Iterate through the subdirectories
        for gtv in ['GTV-T', 'GTV-N']:
            patient_results = {}
            for subdir in subdirs:
                # Obtain the UED summary.json file path
                ued_path = os.path.join(base_dir, subdir, gtv+"_union", "summary.json")
                # Obtain the FND and FPD summary.json file path
                
                fpnd_path = os.path.join(base_dir, subdir, gtv, "summary.json")
                # Check if the summary.json files exist
                if os.path.exists(ued_path) and os.path.exists(fpnd_path):
                    # Read the UED summary.json file
                    with open(ued_path) as f:
                        ued_data = json.load(f)
                    # Read the FND and FPD summary.json file
                    with open(fpnd_path) as f:
                        fpnd_data = json.load(f)

                    # Iterate through the patients in the UED summary.json file
                    for patient_data in ued_data["results"]["all"]:
                        # Obtain the unique patient ID
                        patient_id = os.path.basename(patient_data["reference"]).split(".nii.gz")[0]
                        # Check if the patient ID already exists in the patient_results dictionary
                        if patient_id not in patient_results:
                            # Create a new dictionary for the patient ID
                            patient_results[patient_id] = {}
                        # Obtain the UED value for the patient
                        ued_value = patient_data["1"]["Dice"]
                        ued_volume = patient_data["1"]["Total Positives Test"]

                        # Check if the UED value is higher than the current value for the patient
                        if "UED" not in patient_results[patient_id]:
                            # Add the UED value to the patient's dictionary
                            patient_results[patient_id]["UED"]= ued_value
                            patient_results[patient_id]["UED_volume"]= ued_volume

                            patient_results[patient_id]["UED_th"]= subdir
                        if ued_value > patient_results[patient_id]["UED"]:
                            # Add the UED value to the patient's dictionary
                            patient_results[patient_id]["UED"] = ued_value
                            patient_results[patient_id]["UED_volume"]= ued_volume
                            patient_results[patient_id]["UED_th"] = subdir
                        if ued_volume == 0:
                            patient_results[patient_id]["UED"]= np.nan
                            patient_results[patient_id]["UED_volume"]= ued_volume
                            patient_results[patient_id]["UED_th"]= 0

                    for patient_data in fpnd_data["results"]["all"]:
                        # Obtain the unique patient ID
                        patient_id = os.path.basename(patient_data["reference"]).split(".nii.gz")[0]
                        if patient_id not in patient_results:
                            print("fError, some thing went wrong {patient_id}")
                            # Create a new dictionary for the patient ID
                            break
                        # Obtain the FND value for the patient
                        fnd_value = patient_data["1"]["Dice"]
                        fnd_volume = patient_data["1"]["Total Positives Test"]
                        # Obtain the FPD value for the patient
                        fpd_value = patient_data["2"]["Dice"]
                        fpd_volume = patient_data["2"]["Total Positives Test"]

                        # Check if the FND value is higher than the current value for the patient
                        if "FND" not in patient_results[patient_id]:
                            # Add the FND value to the patient's dictionary
                            patient_results[patient_id]["FND"] = fnd_value
                            patient_results[patient_id]["FND_volume"]= fnd_volume
                            patient_results[patient_id]["FND_th"] = subdir
                        if fnd_value > patient_results[patient_id]["FND"]:
                            # Add the FND value to the patient's dictionary
                            patient_results[patient_id]["FND"]= fnd_value
                            patient_results[patient_id]["FND_volume"]= fnd_volume
                            patient_results[patient_id]["FND_th"] = subdir
                        if fnd_volume == 0:
                            patient_results[patient_id]["FND"]= np.nan
                            patient_results[patient_id]["FND_volume"]= fnd_volume
                            patient_results[patient_id]["FND_th"]= 0

                        # Check if the FPD value is higher than the current value for the patient
                        if "FPD" not in patient_results[patient_id]:
                            # Add the FPD value to the patient's dictionary
                            patient_results[patient_id]["FPD"] = fpd_value
                            patient_results[patient_id]["FPD_volume"]= fpd_volume
                            patient_results[patient_id]["FPD_th"]= subdir
                        if fpd_value > patient_results[patient_id]["FPD"]:
                            # Add the FPD value to the patient's dictionary
                            patient_results[patient_id]["FPD"] = fpd_value
                            patient_results[patient_id]["FPD_volume"]= fpd_volume
                            patient_results[patient_id]["FPD_th"]= subdir
                        if fpd_volume == 0:
                            patient_results[patient_id]["FPD"]= np.nan
                            patient_results[patient_id]["FPD_volume"]= fpd_volume
                            patient_results[patient_id]["FPD_th"]= 0

                        patient_results[patient_id]["GTV"] = gtv
                        patient_results[patient_id]["Ucertainty estimation methods"] = group
            results.append(patient_results)
        dft = pd.DataFrame(results[0]).T
        dfn = pd.DataFrame(results[1]).T
        dft.index.names = ['PatientID']
        dfn.index.names = ['PatientID']
        dft= dft.reset_index(drop=False)
        dfn= dfn.reset_index(drop=False)

        df_ued = pd.concat([dft, dfn])
        df_ued= df_ued.reset_index(drop=True)

        if index == 0:
            df_ued_all = df_ued
        else:
            df_ued_all = pd.concat([df_ued_all, df_ued], axis=0)
            df_ued_all= df_ued_all.reset_index(drop=True)

    return df_ued_all
    
df_ued_all_internal = compute_best_ued(studies)

DROP2_UREGIONS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/u_regions/Task711_NKI_origin/imagesAll_ct_correct"
NNUNET_UREGIONS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/u_regions/Task711_NKI_origin/imagesAll_ct_correct"
PHISEG_UREGIONS_EXT="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/u_regions/Task711_NKI_origin/imagesAll_ct_correct"

studies_ext = [
join(NNUNET_UREGIONS_EXT,'f0_tta'),
join(NNUNET_UREGIONS_EXT,'f0'),
join(DROP2_UREGIONS_EXT,'f0_mc10_tta')
,join(NNUNET_UREGIONS_EXT,'f01234_tta')
#,join(NNUNET_PMAPS,'f012456_tta')
,join(DROP2_UREGIONS_EXT,'f0_tta_snap')
,join(DROP2_UREGIONS_EXT,'f01234_mc10_tta_snap')
,join(PHISEG_UREGIONS_EXT,'f01234_tta')
]
df_ued_all_external = compute_best_ued(studies_ext)
df_ued_all_external

,PatientID,UED,UED_volume,UED_th,FND,FND_volume,FND_th,FPD,FPD_volume,FPD_th,GTV,Ucertainty estimation methods
0,NKI_002,0.139333,11878,7,0.06278,11085,7,0.155526,793,7,GTV-T,Baseline
1,NKI_003,0.270218,5972,7,0.23644,4295,7,0.121432,1677,7,GTV-T,Baseline
2,NKI_005,0.407866,6621,7,0.32537,4966,7,0.11124,1655,7,GTV-T,Baseline
3,NKI_007,0.323929,5541,7,0.21167,3951,7,0.206078,1590,7,GTV-T,Baseline
4,NKI_010,NaN,0,0,NaN,0,0,NaN,0,0,GTV-T,Baseline
...,...,...,...,...,...,...,...,...,...,...,...,...
3299,NKI_333,0.369438,12535,7,0.338453,9150,7,0.436776,3385,7,GTV-N,PhiSeg
3300,NKI_334,0.0,5098,7,0.0,5098,7,NaN,0,0,GTV-N,PhiSeg
3301,NKI_336,0.309284,6902,7,0.315542,5064,7,0.259931,1838,7,GTV-N,PhiSeg
3302,NKI_338,0.388861,26730,7,0.270884,19457,7,0.602775,7273,7,GTV-N,PhiSeg


In [44]:

exp= 'UED'
exp= 'FND'
exp= 'FPD'

print_tests(df_ued_all_internal, exp)

FPD
GTV-T
pvalues before correction:  [8.705748279252395e-16, 0.3541626820157725, 3.288894601514604e-08, 5.475571359566826e-08, 7.184282003004013e-10, 4.3319147819820525e-11]
pvalues after correction: [1, 5.223448967551426e-15, 0.3541626820157725, 9.866683480038985e-08, 1.0951142419314835e-07, 2.8737127981047705e-09, 2.1659573908033716e-10]
0.32(0.29-0.34)
0.21(0.20-0.23)***
0.34(0.31-0.36)
0.36(0.33-0.38)***
0.39(0.36-0.41)***
0.40(0.37-0.43)***
0.44(0.42-0.47)***
GTV-N
pvalues before correction:  [2.0191561033704332e-10, 0.19868771777690175, 0.00013254110554204298, 0.016051776293250955, 6.806173276248683e-05, 2.9203693479754667e-10]
pvalues after correction: [1, 1.2114936614107111e-09, 0.19868771777690175, 0.0003975706175205228, 0.03184589306433334, 0.0002722191379142826, 1.4601846731348776e-09]
0.34(0.30-0.38)
0.24(0.21-0.27)***
0.33(0.30-0.37)
0.36(0.33-0.40)***
0.36(0.33-0.39)*
0.38(0.35-0.41)***
0.47(0.43-0.51)***


In [47]:
exp= 'UED'
exp= 'FND'
exp= 'FPD'

print_tests(df_ued_all_external, exp)

FPD
GTV-T
pvalues before correction:  [5.7119815668003955e-11, 1.4922206363698853e-33, 3.3244251693779676e-36, 3.302977159797594e-38, 4.3152717533031645e-37, 4.3771866414248054e-33]
pvalues after correction: [1, 5.7119815668003955e-11, 4.476661909109656e-33, 1.329770067751187e-35, 1.981786295878556e-37, 2.1576358766515823e-36, 8.754373282849611e-33]
0.16(0.15-0.17)
0.22(0.21-0.24)***
0.34(0.32-0.36)***
0.32(0.30-0.33)***
0.36(0.34-0.38)***
0.37(0.35-0.39)***
0.37(0.35-0.40)***
GTV-N
pvalues before correction:  [2.46237267172169e-07, 2.772638228005272e-26, 1.6277053974326143e-31, 1.5922660989272084e-31, 3.536497583414815e-30, 2.213711843389758e-28]
pvalues after correction: [1, 2.46237267172169e-07, 5.545276456010544e-26, 9.55359659356325e-31, 9.55359659356325e-31, 1.414599033365926e-29, 6.641135530169274e-28]
0.18(0.17-0.19)
0.23(0.21-0.25)***
0.34(0.32-0.36)***
0.35(0.33-0.37)***
0.38(0.36-0.40)***
0.37(0.35-0.39)***
0.44(0.41-0.46)***


## Correlation statistics

In [2]:
import SimpleITK as sitk
import numpy as np
from multiprocessing import Pool
from medpy.metric.binary import hd, asd
from scipy.ndimage import binary_erosion
from src.utils import save_json, subfiles, join, maybe_mkdir_p, reconstruct_seg_df_from_json, reconstruct_calib_df_from_json, reconstruct_UED_df_from_json

def calcualte_target_entropy(arguments, only_seg_roi=False):
    th = 0.7
    umap_folder , seg_folder,  filename = arguments

    # create an dict for data storage

    seg_path = join(seg_folder, filename) 
    umap_path = join(umap_folder, filename.replace(".nii.gz", ".npz")) 
    pid = filename.replace(".nii.gz", "")
    
    
    seg = sitk.GetArrayFromImage(sitk.ReadImage(seg_path))
    umap = np.load(umap_path)['umap']

    resutl_list = []

    for target in range(1,3):

        entropy_results = {}
        entropy_results['PatientID'] = pid

        if target == 1:
            gtv = 'GTV-T'
            
        elif target == 2:
            gtv = 'GTV-N'

        # create an dict for each GTV
        entropy_results['GTV'] = gtv
    
        mask_seg = seg==target

        target_umap = umap[target]
        mask_umap = target_umap>th

        
        roi = mask_seg

        if not only_seg_roi:
            if np.sum(mask_seg) != 0:
                union_roi = (mask_seg + mask_umap)
                union_roi[union_roi>0] = 1
                roi = union_roi
        

        seg_entropy = target_umap * roi
        seg_entropy = seg_entropy[seg_entropy> 0] 



        if np.sum(mask_seg) == 0:
            # no segmentation made for this target (mostly GTV-N)
            entropy_results["Total Entropy"] = np.nan 
            entropy_results["Mean Entropy"] = np.nan 
            entropy_results["Entropy STD"] = np.nan 
            entropy_results["Entropy Volume"] = np.nan 
            entropy_results["Entropy Coefficient of Variation"] = np.nan 
            entropy_results["Logarithm Entropy Coefficient of Variation"] = np.nan 
            # entropy_results["Uncertainty-Segmentation Hausdorff distance"] = np.nan 
            # entropy_results["Uncertainty-Segmentation Mean Surface Distance"] = np.nan 

        else:
            # eros_seg  = binary_erosion(mask_seg, iterations=2).astype(int)
            # seg_line = mask_seg.astype(int) - eros_seg
            # e_hd  = hd(mask_umap, seg_line)
            # e_asd = asd(mask_umap, seg_line)
            entropy_mean = np.mean(seg_entropy) 
            entropy_std = np.std(seg_entropy) 
            entropy_results["Total Entropy"] = np.sum(seg_entropy)
            entropy_results["Mean Entropy"] = entropy_mean
            entropy_results["Entropy STD"] = entropy_std
            entropy_results["Entropy Volume"] = np.sum(roi) 
            entropy_results["Entropy Coefficient of Variation"] = entropy_std / (entropy_mean  + 1e-6) # + 1e-6 to avoid divide by 0
            entropy_results["Logarithm Entropy Coefficient of Variation"] = np.log(entropy_std / (entropy_mean  + 1e-6)) 
            # entropy_results["Uncertainty-Segmentation Hausdorff distance"] = e_hd
            # entropy_results["Uncertainty-Segmentation Mean Surface Distance"] = e_asd

        resutl_list.append(entropy_results)

    return resutl_list

## directorys for internal  probability maps 

DROP2_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
NNUNET_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
PH_PMAPS="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"
PH_GAMM_PMAP="/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_PhiSeg_gamma__nnUNetPlansv2.1/prob_maps/Task901_AUH/imagesTs"


groups = {'Baseline': join(NNUNET_PMAPS,'f0_tta')
          ,'No TTA': join(NNUNET_PMAPS,'f0')
          ,'MC Dropout': join(DROP2_PMAPS,'f0_mc10_tta')
          ,'Ensemble': join(NNUNET_PMAPS,'f01234_tta')
          ,'Snapshots':join(DROP2_PMAPS,'f0_tta_snap')
          ,"Complex":join(DROP2_PMAPS,'f01234_mc10_tta_snap')
          ,'PhiSeg': join(PH_PMAPS,'f01234_tta')
          }


In [ ]:

Task = 'Complex' #Complex #PhiSeg #External #Baseline #Snapshots
    ## complex
seg_folder = groups[Task]
umap_folder = seg_folder.replace("prob_maps","umaps")

## external
# seg_folder = '/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/prob_maps/Task711_NKI_origin/imagesAll_ct_correct/f0_mc10_tta' #f01234_mc10_tta'
# umap_folder =  '/data/jintao/nnUNet/nnUNet_results/nnUNet/3d_fullres/Task901_AUH/nnUNetTrainerV2_dropout__nnUNetPlansv2.1/umaps/Task711_NKI_origin/imagesAll_ct_correct/f0_mc10_tta' #f01234_mc10_tta'

seg_files_in = subfiles(seg_folder, suffix=".nii.gz", join=False) 
umaps = [umap_folder] * len(seg_files_in)
segs = [seg_folder] * len(seg_files_in)

p = Pool(128)
entropy_results=  p.map(calcualte_target_entropy, zip(umaps, segs, seg_files_in))
p.close()
p.join()
#=========================================#
entropy_t = []
entropy_n = []
for result in entropy_results:
    entropy_t.append(result[0])
    entropy_n.append(result[1])
entropy_dft = pd.DataFrame(entropy_t)
entropy_dfn = pd.DataFrame(entropy_n)

entropy_df_all = pd.concat([entropy_dft, entropy_dfn], axis=0)
entropy_df_all = entropy_df_all.reset_index(drop=False)

complex_all_data = all_seg_internal[all_seg_internal["Ucertainty estimation methods"]==Task]
complex_all_data_entropy = pd.merge(complex_all_data, entropy_df_all, how="inner", on=["PatientID", "GTV"])
